In [1]:
import talib
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2017-2018.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
def add_fractal(df, i, look_back=3):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan


def add_trade(df, i):
    if i < 4:
        return np.nan, np.nan, np.nan

    ema200 = df.loc[df.index[i - 1], 'ema200']
    curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
    curr_mh = df.loc[df.index[i - 1], 'Mid_High']
    macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
    macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
    curr_ao = df.loc[df.index[i], 'Ask_Open']
    curr_bo = df.loc[df.index[i], 'Bid_Open']

    if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml > ema200:
        pullback = None
        u = i - 4

        while u >= 0:
            if df.loc[df.index[u], 'fractal'] == 1:
                curr_pullback = float(df.loc[df.index[u], 'Bid_Low']) - 0.0010
                curr_fractal_distance = curr_ao - curr_pullback

                if curr_fractal_distance <= 0.0100:
                    pullback = curr_pullback
                    break

            u -= 1

        if pullback is not None:
          stop_loss = round(pullback, 5)

          if stop_loss < curr_ao:
            curr_pips_to_risk = curr_ao - stop_loss
            stop_gain = round(curr_ao + (curr_pips_to_risk * 1.5), 5)

            return 1, stop_loss, stop_gain

        return 0, 0, 0

    elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh < ema200:
        pullback = None
        u = i - 4

        while u >= 0:
            if df.loc[df.index[u], 'fractal'] == 2:
                curr_pullback = float(df.loc[df.index[u], 'Ask_High']) + 0.0010
                curr_fractal_distance = curr_pullback - curr_bo

                if curr_fractal_distance <= 0.0100:
                    pullback = curr_pullback
                    break

            u -= 1

        if pullback is not None:
          stop_loss = round(pullback, 5)

          if stop_loss > curr_bo:
            curr_pips_to_risk = stop_loss - curr_bo
            stop_gain = round(curr_bo - (curr_pips_to_risk * 1.5), 5)

            return 2, stop_loss, stop_gain

        return 0, 0, 0

    else:
        return 0, 0, 0

In [5]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
trade_infos = [add_trade(df, i) for i in range(df.shape[0])]
trades = []
stop_losses = []
stop_gains = []

for trade, stop_loss, stop_gain in trade_infos:
    trades.append(trade)
    stop_losses.append(stop_loss)
    stop_gains.append(stop_gain)

df['trade'], df['stop_loss'], df['stop_gain'] = trades, stop_losses, stop_gains
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['fractal'].value_counts())
print(df['trade'].value_counts())

0.0    19615
1.0     2356
2.0     2266
Name: fractal, dtype: int64


In [7]:
def plot_candles(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.Date, open=df_plot.Mid_Open, high=df_plot.Mid_High, low=df_plot.Mid_Low, close=df_plot.Mid_Close,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ))

    # df_fractals = df_plot[df_plot.fractal != 0]
    # df_fractal_prices = [df_fractals.loc[df_fractals.index[i], 'Mid_Low'] if df_fractals.loc[df_fractals.index[i], 'fractal'] == 1 else df_fractals.loc[df_fractals.index[i], 'Mid_High'] for i in range(len(df_fractals))]

    # fig.add_trace(go.Scatter(
    #     x=df_fractals.Date,
    #     y=df_fractal_prices,
    #     marker=dict(color="crimson", size=10),
    #     name='Fractal',
    #     mode='markers'
    #     ))

    df_sl_sg = df_plot[df_plot.trade != 0]

    fig.add_trace(go.Scatter(
        x=df_sl_sg.Date,
        y=df_sl_sg.stop_loss,
        marker=dict(color="crimson", size=10),
        name='Stop Loss',
        mode='markers'
        ))

    fig.add_trace(go.Scatter(
        x=df_sl_sg.Date,
        y=df_sl_sg.stop_gain,
        marker=dict(color="green", size=10),
        name='Stop Gain',
        mode='markers'
        ))

    df_buys = df_plot[df_plot.trade == 1]
    df_sells = df_plot[df_plot.trade == 2]

    fig.add_trace(go.Candlestick(
        x=df_buys.Date, open=df_buys.Mid_Open, high=df_buys.Mid_High, low=df_buys.Mid_Low, close=df_buys.Mid_Close,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='blue',
        decreasing_fillcolor='blue',
        increasing_line_color='blue',
        decreasing_line_color='blue',
        name='Buy'
    ))

    fig.add_trace(go.Candlestick(
        x=df_sells.Date, open=df_sells.Mid_Open, high=df_sells.Mid_High, low=df_sells.Mid_Low, close=df_sells.Mid_Close,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='orange',
        decreasing_fillcolor='orange',
        increasing_line_color='orange',
        decreasing_line_color='orange',
        name='Sell'
    ))

    fig.add_trace(go.Scatter(
        x=df_plot.Date,
        y=df_plot.ema200,
        line=dict(width=2, color='purple'),
        line_shape='spline',
        name='EMA',
        mode='lines'
    ))

    fig.update_layout(width=1400,height=600,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")

    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )

    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )

    macd_fig = go.Figure()

    macd_fig.add_trace(go.Scatter(
        x=df_plot.Date,
        y=df_plot.macdsignal,
        line=dict(width=2, color='#03adfc'),
        line_shape='spline',
        name='SIGNAL',
        mode='lines'
    ))

    macd_fig.add_trace(go.Scatter(
        x=df_plot.Date,
        y=df_plot.macd,
        line=dict(width=2, color='#fc4a03'),
        line_shape='spline',
        name='MACD',
        mode='lines'
    ))

    macd_fig.update_layout(width=1400,height=200,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")

    macd_fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )

    macd_fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )

    fig.show()
    macd_fig.show()

In [10]:
plot_candles(df.iloc[50:150, :])

In [ ]:
len(df)

In [ ]:
df.Date[0]